In [1]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

In [36]:
pisa_train = pd.read_csv('D:/DS/analytics_edge/2. Linear Regression/pisa2009train.csv')
pisa_test = pd.read_csv('D:/DS/analytics_edge/2. Linear Regression/pisa2009test.csv')

In [37]:
pisa_train.shape

(3663, 24)

In [38]:
pisa_test.shape

(1570, 24)

In [39]:
pisa = pisa_train.append(pisa_test, ignore_index = True)

In [40]:
pisa.shape

(5233, 24)

In [7]:
pandas_profiling.ProfileReport(pisa)

Number of variables,24
Number of observations,5233
Total Missing (%),3.4%
Total size in memory,981.3 KiB
Average record size in memory,192.0 B
Numeric,20
Categorical,1
Boolean,3
Date,0
Text (Unique),0
Rejected,0


In [158]:
pisa_train.readingScore.mean()

497.91140322140376

In [159]:
pisa_train.groupby(['male']).mean()

,grade,preschool,expectBachelors,motherHS,motherBachelors,motherWork,fatherHS,fatherBachelors,fatherWork,selfBornUS,...,englishAtHome,computerForSchoolwork,read30MinsADay,minutesPerWeekEnglish,studentsInEnglish,schoolHasLibrary,publicSchool,urban,schoolSize,readingScore
male,,,,,,,,,,,,,,,,,,,,,
0,10.145170,0.715986,0.818647,0.869863,0.312150,0.739703,0.852959,0.295320,0.841980,0.925441,...,0.87500,0.904141,0.391892,264.593329,24.683619,0.961828,0.953099,0.366834,1357.814620,512.940631
1,10.036859,0.729246,0.754615,0.889746,0.382902,0.729418,0.865330,0.367153,0.863662,0.936854,...,0.86845,0.894823,0.192121,267.769231,24.318630,0.973199,0.915598,0.402244,1380.298716,483.532479


In [160]:
pisa.shape

(5233, 24)

In [161]:
pisa_new = pisa.dropna()

In [162]:
pisa_new.shape

(3404, 24)

In [163]:
pisa_new.sample(5)

,grade,male,raceeth,preschool,expectBachelors,motherHS,motherBachelors,motherWork,fatherHS,fatherBachelors,...,englishAtHome,computerForSchoolwork,read30MinsADay,minutesPerWeekEnglish,studentsInEnglish,schoolHasLibrary,publicSchool,urban,schoolSize,readingScore
4432,10,1,Hispanic,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,300.0,30.0,1.0,1,0,1900.0,470.79
4329,11,1,Asian,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,1.0,0.0,260.0,25.0,1.0,0,1,570.0,480.81
4121,10,1,More than one race,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,225.0,15.0,1.0,1,0,1380.0,467.26
1979,11,1,Hispanic,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,264.0,30.0,1.0,1,0,2402.0,507.75
4297,10,1,American Indian/Alaska Native,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,450.0,20.0,1.0,1,1,1062.0,554.73


Linear Regression with categorical varaibles.

Raceeth has unrdered factor variables:

* American Indian/Alaska Native
* Asian
* Black
* Hispanic
* More than one race
* Native Hawaiian/Other Pacific Islander
* White

In [164]:
pisa_new['raceeth'].value_counts()

White                                     2083
Hispanic                                   701
Black                                      307
Asian                                      140
More than one race                         116
American Indian/Alaska Native               31
Native Hawaiian/Other Pacific Islander      26
Name: raceeth, dtype: int64

White is chosen as the reference variable

New column added with binary values based on race.

In [165]:
pisa_raceeth = pd.get_dummies(pisa_new.raceeth)

In [166]:
pisa_raceeth.head()

,American Indian/Alaska Native,Asian,Black,Hispanic,More than one race,Native Hawaiian/Other Pacific Islander,White
1,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0
4,0,0,0,1,0,0,0
7,0,0,0,0,0,0,1
9,0,0,0,0,1,0,0


Appending the column of pisa_raceeth to pisa_new.

In [167]:
pisa_new = pisa_new.join(pisa_raceeth)

In [168]:
pisa_new.head()

,grade,male,raceeth,preschool,expectBachelors,motherHS,motherBachelors,motherWork,fatherHS,fatherBachelors,...,urban,schoolSize,readingScore,American Indian/Alaska Native,Asian,Black,Hispanic,More than one race,Native Hawaiian/Other Pacific Islander,White
1,11,1,White,0.0,0.0,1.0,1.0,1.0,1.0,0.0,...,0,1173.0,575.01,0,0,0,0,0,0,1
3,10,0,Black,1.0,1.0,0.0,0.0,1.0,1.0,0.0,...,1,2640.0,458.11,0,0,1,0,0,0,0
4,10,1,Hispanic,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,1,1095.0,613.89,0,0,0,1,0,0,0
7,10,0,White,1.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0,1913.0,439.36,0,0,0,0,0,0,1
9,10,1,More than one race,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0,899.0,465.90,0,0,0,0,1,0,0


In [169]:
X = pd.DataFrame(pisa_new.drop('readingScore', axis = 1))

In [170]:
X = X.drop('raceeth', axis = 1)

In [171]:
X.head()

,grade,male,preschool,expectBachelors,motherHS,motherBachelors,motherWork,fatherHS,fatherBachelors,fatherWork,...,publicSchool,urban,schoolSize,American Indian/Alaska Native,Asian,Black,Hispanic,More than one race,Native Hawaiian/Other Pacific Islander,White
1,11,1,0.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,1,0,1173.0,0,0,0,0,0,0,1
3,10,0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,1,1,2640.0,0,0,1,0,0,0,0
4,10,1,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,1,1,1095.0,0,0,0,1,0,0,0
7,10,0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,...,1,0,1913.0,0,0,0,0,0,0,1
9,10,1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,...,1,0,899.0,0,0,0,0,1,0,0


In [172]:
y = pd.DataFrame(pisa_new.readingScore)

In [173]:
y.head()

,readingScore
1,575.01
3,458.11
4,613.89
7,439.36
9,465.90


In [174]:
X.shape

(3404, 29)

In [175]:
y.shape

(3404, 1)

In [176]:
linreg = LinearRegression()
feature_selector = RFE(linreg, 5, step = 1)
feature_selector = feature_selector.fit(X, y)
feature_names = list(X.columns)
print("Features sorted by their rank:")
print(sorted(zip(map(lambda x: round(x, 4), feature_selector.ranking_), feature_names)))

Features sorted by their rank:
[(1, 'Asian'), (1, 'Native Hawaiian/Other Pacific Islander'), (1, 'White'), (1, 'expectBachelors'), (1, 'read30MinsADay'), (2, 'grade'), (3, 'American Indian/Alaska Native'), (4, 'Black'), (5, 'fatherBachelors'), (6, 'computerForSchoolwork'), (7, 'More than one race'), (8, 'motherBachelors'), (9, 'male'), (10, 'publicSchool'), (11, 'fatherHS'), (12, 'englishAtHome'), (13, 'motherBornUS'), (14, 'Hispanic'), (15, 'motherHS'), (16, 'fatherWork'), (17, 'urban'), (18, 'motherWork'), (19, 'selfBornUS'), (20, 'fatherBornUS'), (21, 'schoolHasLibrary'), (22, 'preschool'), (23, 'minutesPerWeekEnglish'), (24, 'studentsInEnglish'), (25, 'schoolSize')]


C:\Users\Atheesh Krishnan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [197]:
X_new = pd.DataFrame(X.Asian)
X_new = X_new.join(X['Native Hawaiian/Other Pacific Islander'])
X_new = X_new.join(X['White'])
X_new = X_new.join(X['expectBachelors'])
X_new = X_new.join(X['read30MinsADay'])
X_new = X_new.join(X['grade'])
X_new = X_new.join(X['American Indian/Alaska Native'])
X_new = X_new.join(X['Black'])
X_new = X_new.join(X['fatherBachelors'])
X_new.shape

(3404, 9)

In [190]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size = 0.5, random_state = 0)

In [191]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)
linreg.score(X_test, y_test)

0.30386027347679656

In [192]:
linreg.coef_.mean()

19.750741919271874

In [193]:
linreg.intercept_

array([170.03544578])

In [194]:
y_pred = linreg.predict(X_test)

In [195]:
print(mean_absolute_error(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))
print(np.sqrt(mean_squared_error(y_test, y_pred)))
print(r2_score(y_test, y_pred))

58.34037690740738
5425.020065573562
73.65473552714423
0.30386027347679656


In [196]:
y_pred_df = pd.DataFrame(y_pred)
y_pred_df.head()

,0
0,549.735916
1,493.959080
2,494.055013
3,523.449437
4,460.787014


In [198]:
y_pred.mean()

518.3574057540013